In [6]:
%load_ext autoreload
%autoreload 2

import os
import json
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import ray_results_interpreter as rri
import subprocess
import concurrent.futures
from main_run import MainRun

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
mode = "train"
setting_name = 'serial_system_hard'

models = []

# base_path = "/user/ml4723/Prj/NIC/ray_results/generic_architecture_serial_hard/GNN_MP/run_2025-02-06_09-25-45"
# for subdir in os.listdir(base_path):
#     subdir_path = os.path.join(base_path, subdir)
#     if os.path.isdir(subdir_path):
#         model_path = os.path.join(subdir_path, "model_0.pt")
#         if os.path.exists(model_path):
#             models.append(model_path)

base_path = "/user/ml4723/Prj/NIC/ray_results/generic_architecture_serial_hard/GNN_MP_bottleneck_loss_stop_gradient_L1_skip_connection/run_2025-02-07_14-59-26/run_08745_00001_1_config=serial_system_hard,dev_batch_size=16,dev_n_samples=16,early_stop_check_epochs=50,gradient_clipping_norm_v_2025-02-07_14-59-26"
for file in os.listdir(base_path):
    if file.endswith(".pt"):
        model_path = os.path.join(base_path, file)
        models.append(model_path)


def run_main_run(model_path):
    try:
        # Extract hyperparam name from model path
        hyperparam_name = model_path.split('/')[7]
        print(f"Running main_run.py for path {model_path}")
        cmd = [
            "/user/ml4723/.conda/envs/neural_inventory_control/bin/python",
            "main_run.py",
            mode, # test or test_on_dev
            setting_name,
            hyperparam_name,
            model_path,
        ]
        env = {
            **os.environ,
            "MKL_THREADING_LAYER": "GNU",
            "MKL_SERVICE_FORCE_INTEL": "1"
        }
        subprocess.run(cmd, capture_output=True, text=True, check=True, env=env, cwd="/user/ml4723/Prj/NIC/")
    except subprocess.CalledProcessError as e:
        print(f"Error running main_run.py for path {model_path}: {e}")
        print(f"Error output: {e.stderr}")
    except Exception as e:
        print(f"Unexpected error running main_run.py for path {model_path}: {e}")

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for path in models:
        while len(futures) >= 18:
            done, not_done = concurrent.futures.wait(
                futures, 
                return_when=concurrent.futures.FIRST_COMPLETED
            )
            futures = list(not_done)
        futures.append(executor.submit(run_main_run, path))
    concurrent.futures.wait(futures)


Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/generic_architecture_serial_hard/GNN_MP_bottleneck_loss_stop_gradient_L1_skip_connection/run_2025-02-07_14-59-26/run_08745_00001_1_config=serial_system_hard,dev_batch_size=16,dev_n_samples=16,early_stop_check_epochs=50,gradient_clipping_norm_v_2025-02-07_14-59-26/model_2600.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/generic_architecture_serial_hard/GNN_MP_bottleneck_loss_stop_gradient_L1_skip_connection/run_2025-02-07_14-59-26/run_08745_00001_1_config=serial_system_hard,dev_batch_size=16,dev_n_samples=16,early_stop_check_epochs=50,gradient_clipping_norm_v_2025-02-07_14-59-26/model_1450.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/generic_architecture_serial_hard/GNN_MP_bottleneck_loss_stop_gradient_L1_skip_connection/run_2025-02-07_14-59-26/run_08745_00001_1_config=serial_system_hard,dev_batch_size=16,dev_n_samples=16,early_stop_check_epochs=50,gradient_clipping_norm_v_2025-02-07_14-